In [1]:
"""
Script defining WfieldDataSet Class and loaders to be used along with VAE model.
"""
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from wfield import * #for loading wfield data in nice format
from pathlib import Path

In [2]:
class WfieldDataSet(Dataset):
    """
    Defines WfieldDataSet to be used with VAE model.
    """
    def __init__(self, file_path, transform=None):
        #use wfield to create a memory map version of entire data
        print('Dataclass_here')
        data = mmap_dat(file_path)
        print('Dataclass_here1',np.shape(data))
        print('Dataclass_here1',np.shape(data[0:100]))
        #data = data[0:100]
        self.df = data
        
#         self.max = np.amax(data)
#         self.min = np.amin(data)
#         self.mean = np.mean(data)             
        self.max = np.max(data)
        self.min = np.min(data)
        self.mean = np.mean(data)

#.      self.max = np.max(data)
        self.transform = transform
        print('Dataclass_here6')
    def __len__(self):
        """
        Returns number of samples in dset.
        """
        return (int(self.df.shape[0] - 7))
    def __getitem__(self, idx):
        """
        Returns a single sample from dset.
        """
        time_start = idx
        time_end = time_start + 7
        frame = self.df[time_start:time_end, :, :, :]
        scld_frame = np.true_divide((frame - self.min), (self.max - self.min)) #min/max norm (global)
        sample = {'frame': scld_frame}
        if self.transform:
            sample = self.transform(sample)
        return sample

class ToTensor(object):
    """
    Converts sample arrays to tensor which can be directly fed to model
    """
    def __call__(self, sample):
        frame = sample['frame']
        return {'frame':torch.from_numpy(frame).float()}

def setup_data_loaders(batch_size=32, shuffle=(True, False), file_path=''):
    #for now am not worried about train/test splitting
    print('Dataclass_here3')
    dset = WfieldDataSet(file_path=file_path, transform = ToTensor())
    print('Dataclass_here4')
    train_loader = DataLoader(dset, batch_size=batch_size, \
    shuffle=shuffle[0], num_workers = 0)
    test_loader = DataLoader(dset, batch_size=batch_size, \
    shuffle=shuffle[1], num_workers=0)
    print('Dataclass_here5')
    return{'train':train_loader, 'test':test_loader}